In [1]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.max_columns = 120

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  np = pd.np
/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and wil

In [2]:
sms = get_data('sms-spam')
index = ['sms{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(sms)), sms.spam)]
sms.index = index
sms.head(6)
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=sms.text).toarray()
len(tfidf.vocabulary_)

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


9232

#### 隐形狄利克雷分布原理
- 设想一台自动生成文档的机器，只有两个选项来控制生成文档的两个属性
    - 生成文档的词的数量（泊松分布）
    - 文档中混合的主题的数量（狄利克雷分布）
- 除此之外，最关键的问题在于确定一个主题-词项矩阵，该矩阵表示了每个词对主题的贡献权重。一但能够确定这个矩阵，机器就可以在选择好的主题上反复迭代选择词，直到生成一篇足够长度的文档。
- 回到对现有文档估计主题的问题上来，LDiA可以用于关于词和主题的参数，Blei和Ng通过分析语料库中文档的统计数据确定两个参数
    - 第一个参数可以通过计算语料库的平均词（n-gram）数量、
    - 第二个参数更棘手，必须先猜测有几个主题数再为主题分配语词，最后优化目标函数
-  “LSA试图将原本分散的东西分散开，LDiA试图将原本接近的东西接近在一起”